In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
import gradio as gr
import json

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('float_format', '{:f}'.format)

In [2]:
resumes = pd.read_csv('dst-3.0_16_1_hh_database.csv', delimiter=';')

In [3]:
resumes.fillna(0, inplace=True)

In [4]:

def load_data(file_path, separator):
    return pd.read_csv(file_path, sep=separator)

# парсим возраст
def parse_age(text):
    match = re.search(r'(\d+) лет', text)
    return int(match.group(1)) if match else None

# город + релокация
def parse_city_relocation(text):
    city = text.split(',')[0]
    relocation = 'не готов к переезду' not in text
    return city, relocation
# заполняем пробелы иначе эластик не взлетит
def handle_missing_data(df):
    
    df.fillna('Unknown', inplace=True)
    return df

def embedding_function(text, embedder):
    embeddings = embedder.encode(text, convert_to_tensor=True)
    #print(embeddings)
    return embeddings

# предобработка резюме
def preprocess_resumes(resumes_df):
    resumes_df['age'] = resumes_df['Пол, возраст'].apply(parse_age)
    resumes_df[['city', 'relocation_ready']] = resumes_df['Город, переезд, командировки'].apply(
        lambda x: pd.Series(parse_city_relocation(x))
    )
    embedder = SentenceTransformer('all-mpnet-base-v2', device='cpu')
    resumes_df['embeddings'] = [embedding_function(text, embedder) for text in resumes_df['Ищет работу на должность:']]
    resumes_df['embeddings1'] = [embedding_function(str(text), embedder) for text in resumes_df['Опыт работы']]
    resumes_df['embeddings2'] = [embedding_function(str(text), embedder) for text in resumes_df['Последнее/нынешнее место работы']]
    resumes_df['embeddings3'] = [embedding_function(str(text), embedder) for text in resumes_df['Последняя/нынешняя должность']]
    resumes_df['sum_embeddings'] = resumes_df[['embeddings', 'embeddings1', 'embeddings2', 'embeddings3']].sum(axis=1)

# Удаление первых четырех колонок
    resumes_df = resumes_df.drop(['embeddings', 'embeddings1', 'embeddings2', 'embeddings3'], axis=1)
    return resumes_df

resumes_df = resumes#load_data('dst-3.0_16_1_hh_database.csv', ';')
zalupa = resumes_df.head(100)
resumes_df = preprocess_resumes(zalupa)

/tmp/ipykernel_10426/3053868893.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumes_df['age'] = resumes_df['Пол, возраст'].apply(parse_age)
/tmp/ipykernel_10426/3053868893.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumes_df[['city', 'relocation_ready']] = resumes_df['Город, переезд, командировки'].apply(
/tmp/ipykernel_10426/3053868893.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [2]:
from pymongo import MongoClient

# Подключение к MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["find_resume"]  # Указываем имя вашей базы данных
resumes_collection = db["resumes"]

In [6]:
resumes_df

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,age,city,relocation_ready,sum_embeddings
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,39.000000,Советск (Калининградская область),False,"[tensor(0.1114), tensor(-0.1322), tensor(0.045..."
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,60.000000,Королев,False,"[tensor(0.1998), tensor(-0.1083), tensor(0.039..."
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,36.000000,Тверь,True,"[tensor(0.1278), tensor(-0.0501), tensor(0.083..."
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,38.000000,Саратов,False,"[tensor(0.0348), tensor(-0.0900), tensor(0.023..."
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано,26.000000,Москва,True,"[tensor(0.1124), tensor(0.0421), tensor(0.0962..."
5,"Мужчина , 29 лет , родился 5 октября 1989",25000 руб.,Технический специалист,"Старый Оскол , не готов к переезду , не готов ...",полная занятость,"полный день, сменный график",Опыт работы 9 лет 9 месяцев Технический специ...,Комбинат хлебопродуктов Старооскольский,Слесарь КИПиА,Неоконченное высшее образование 2013 Воронежс...,22.04.2019 15:59,Имеется собственный автомобиль,29.000000,Старый Оскол,False,"[tensor(0.1162), tensor(-0.1305), tensor(0.056..."
6,"Мужчина , 46 лет , родился 19 сентября 1972",250000 руб.,Руководитель ИТ-проектов,"Москва , не готов к переезду , готов к редким ...",полная занятость,полный день,Опыт работы 22 года 9 месяцев Руководитель ИТ...,"СИБИНТЕК, ООО ИК",Менеджер (Руководитель ИТ-проектов),Высшее образование 2008 ФГОУ ВПО «Уральская ак...,25.04.2019 22:48,Не указано,46.000000,Москва,False,"[tensor(0.1689), tensor(-0.0625), tensor(0.100..."
7,"Мужчина , 29 лет , родился 9 июля 1989",70000 руб.,Инженер АСУ ТП,"Москва , м. Бульвар Рокоссовского , готов к п...",полная занятость,полный день,Опыт работы 3 года 11 месяцев Декабрь 2016 — ...,ФМ-инжиниринг,Инженер АСУ ТП,Высшее образование 2014 Белорусская Государств...,07.05.2019 17:59,Не указано,29.000000,Москва,True,"[tensor(0.0559), tensor(-0.0934), tensor(0.020..."
8,"Мужчина , 29 лет , родился 11 июля 1989",65000 руб.,Ревизор,"Москва , м. Шоссе Энтузиастов , готов к перее...",полная занятость,полный день,Опыт работы 8 лет 9 месяцев Декабрь

In [7]:
resumes_df.to_csv('проведенная_база.csv', index=False)

In [18]:
#resumes_df["sum_embeddings"] = resumes_df["sum_embeddings"].apply(lambda tensor: json.dumps(tensor.tolist()))

data = resumes_df.to_dict(orient="records")

# Вставка данных в MongoDB
resumes_collection.insert_many(data);


In [5]:
resumes_collection
document_count =resumes_collection.count_documents({})

# Вывод результата
print(f'Количество документов в коллекции: {document_count}')


Количество документов в коллекции: 44844


In [20]:
from pymongo import MongoClient

# Подключение к MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["find_resume"]
resumes_collection = db["resumes"]

# Вывод всех документов в коллекции
for document in resumes_collection.find().limit(3):
    print(document)

{'_id': ObjectId('65843e895f6bfb04fbf34ad0'), 'Пол, возраст': 'Мужчина ,  39 лет , родился 27 ноября 1979', 'ЗП': '29000 руб.', 'Ищет работу на должность:': 'Системный администратор', 'Город, переезд, командировки': 'Советск (Калининградская область) , не готов к переезду , не готов к командировкам', 'Занятость': 'частичная занятость, проектная работа, полная занятость', 'График': 'гибкий график, полный день, сменный график, вахтовый метод, удаленная работа', 'Опыт работы': 'Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники', 'Последнее/нынешнее место работы': 'МАОУ "СОШ № 1 г.Немана"', 'Последняя/нынешняя должность': 'Системный администратор', 'Образование

In [ ]:
import json
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from pymongo import MongoClient

def embedding_function(text):
    embedder = SentenceTransformer('all-mpnet-base-v2', device='cpu')
    embeddings = embedder.encode(text, convert_to_tensor=True)
    return embeddings.tolist()

def find_similar_documents(wanna_job, expa, top_k=10):
    # Создаем эмбеддинги для входных данных
    wanna_job_embedding = embedding_function(wanna_job)
    expa_embedding = embedding_function(expa)
    
    # Выравниваем размеры векторов, добавляя нулевые значения, если вектор короче
    max_len = max(len(wanna_job_embedding), len(expa_embedding))
    wanna_job_embedding += [0] * (max_len - len(wanna_job_embedding))
    expa_embedding += [0] * (max_len - len(expa_embedding))
    
    # Усредняем векторы
    input_embedding = [(x + y) / 2 for x, y in zip(wanna_job_embedding, expa_embedding)]
    print(input_embedding)
    # Подключение к MongoDB
    client = MongoClient("mongodb://localhost:27017/")
    db = client["find_resume"]
    resumes_collection = db["resumes"]

    # Поиск близких документов по косинусному расстоянию
    similar_documents = resumes_collection.find(
        {},
        {"sum_embeddings": 1}  # Запрос только столбца с эмбеддингами
    )

    results = []
    for document in similar_documents:
        sum_embeddings = json.loads(document["sum_embeddings"])  # Преобразование из строки JSON в список
        sum_embeddings += [0] * (max_len - len(sum_embeddings))  # Выравниваем размеры векторов
        sum_embeddings = [(x + y) / 2 for x, y in zip(sum_embeddings, input_embedding)]  # Усредняем векторы
        cosine_similarity = np.dot(input_embedding, sum_embeddings) / (np.linalg.norm(input_embedding) * np.linalg.norm(sum_embeddings))
        results.append({"cosine_similarity": cosine_similarity, "document": document})

    # Сортировка по убыванию сходства и возврат топовых результатов
    results = sorted(results, key=lambda x: x["cosine_similarity"], reverse=True)[:top_k]

    return results

if __name__ == "__main__":
    # Пример использования
    wanna_job_input = "Технический писатель"
    expa_input = "Технический писатель"

    similar_documents = find_similar_documents(wanna_job_input, expa_input)

    print("Top 10 схожих документов:")
    for result in similar_documents:
        print(f"Cosine Similarity: {result['cosine_similarity']}")
        print(result['document'])
        print("-" * 50)


In [26]:
import json
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from pymongo import MongoClient

def embedding_function(text):
    embedder = SentenceTransformer('all-mpnet-base-v2', device='cpu')
    embeddings = embedder.encode(text, convert_to_tensor=True)
    return embeddings.tolist()

def  find_similar_documents(wanna_job, expa, top_k=10, city=None, relocation_ready=None):
    # Создаем эмбеддинги для входных данных
    wanna_job_embedding = embedding_function(wanna_job)
    expa_embedding = embedding_function(expa)
    filter_criteria = {}

    if city is not None:
        filter_criteria["Город, переезд, командировки"] = {"$regex": f".*{city}.*", "$options": "i"}

    if relocation_ready is not None:
        filter_criteria["relocation_ready"] = relocation_ready
    # Выравниваем размеры векторов, добавляя нулевые значения, если вектор короче
    max_len = max(len(wanna_job_embedding), len(expa_embedding))
    wanna_job_embedding += [0] * (max_len - len(wanna_job_embedding))
    expa_embedding += [0] * (max_len - len(expa_embedding))
    
    # Усредняем векторы
    input_embedding = [(x + y) / 2 for x, y in zip(wanna_job_embedding, expa_embedding)]

    # Подключение к MongoDB
    client = MongoClient("mongodb://localhost:27017/")
    db = client["find_resume"]
    resumes_collection = db["resumes"]

    # Поиск близких документов по косинусному расстоянию
    similar_documents = resumes_collection.find(
        filter_criteria,
        {"sum_embeddings": 1, "Пол, возраст": 1, "ЗП": 1, "Ищет работу на должность:": 1, 
         "Город, переезд, командировки": 1, "Занятость": 1, "График": 1, "Опыт работы": 1, 
         "Последнее/нынешнее место работы": 1, "Последняя/нынешняя должность": 1, 
         "Образование и ВУЗ": 1, "Обновление резюме": 1, "Авто": 1, "city": 1}
    )

    results = []
    for document in similar_documents:
        sum_embeddings = json.loads(document["sum_embeddings"])  # Преобразование из строки JSON в список
        sum_embeddings += [0] * (max_len - len(sum_embeddings))  # Выравниваем размеры векторов
        sum_embeddings = [(x + y) / 2 for x, y in zip(sum_embeddings, input_embedding)]  # Усредняем векторы
        cosine_similarity = np.dot(input_embedding, sum_embeddings) / (np.linalg.norm(input_embedding) * np.linalg.norm(sum_embeddings))
        result_document = {field: document.get(field, '') for field in ["Пол, возраст", "ЗП", "Ищет работу на должность:", 
                                                                       "Город, переезд, командировки", "Занятость", 
                                                                       "График", "Опыт работы", 
                                                                       "Последнее/нынешнее место работы", 
                                                                       "Последняя/нынешняя должность", 
                                                                       "Образование и ВУЗ", "Обновление резюме", "Авто"]}
        results.append({"cosine_similarity": cosine_similarity, "document": result_document})

    # Сортировка по убыванию сходства и возврат топовых результатов
    results = sorted(results, key=lambda x: x["cosine_similarity"], reverse=True)[:top_k]

    return results

if __name__ == "__main__":
    # Пример использования
    wanna_job_input = "Системный администратор"
    expa_input = "Настройка сети, администрирование сети"

    similar_documents = find_similar_documents(wanna_job_input, expa_input,  city="Москва ", relocation_ready=True)

    print("Top 10 схожих документов:")
    for result in similar_documents:
        print(f"Cosine Similarity: {result['cosine_similarity']}")
        print(result['document'])
        print("-" * 50)


Top 10 схожих документов:
Cosine Similarity: 0.9642494495094501
{'Пол, возраст': 'Мужчина ,  28 лет , родился 17 июля 1990', 'ЗП': '130000 руб.', 'Ищет работу на должность:': 'Старший системный администратор', 'Город, переезд, командировки': 'Москва ,  м. Савеловская , готов к переезду , готов к командировкам', 'Занятость': 'проектная работа, полная занятость', 'График': 'гибкий график, полный день, сменный график, удаленная работа', 'Опыт работы': 'Опыт работы 13 лет 3 месяца  Старший системный администратор 130 000 руб. Информационные технологии, интернет, телеком Системный администратор Занятость: проектная работа, полная занятость График работы: гибкий график, полный день, сменный график, удаленная работа Опыт работы 13 лет 3 месяца Октябрь 2017 — по настоящее время 1 год 7 месяцев НИКИМТ-Атомстрой, ОАО Москва , www.atomrus.ru Строительство, недвижимость, эксплуатация, проектирование ... Строительство коммерческих объектов (торговые площади, офисные здания) Системный администратор 